# Lesson 4.1: Deployment in the Cloud with Seldon & Kubeflow on AWS

<!-- [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zenml-io/zenbytes/blob/main/4-1_Cloud_Deployment.ipynb)  -->

<span style="color:red"><b>Colab Not Supported:</b></span> Deploying on the cloud requires local Docker and Kubernetes installations, which are currently not supported by Colab. Please clone the [ZenBytes repository](https://github.com/zenml-io/zenbytes) locally and follow the lesson there.

***Key Concepts:*** *Cloud Deployment, Kubernetes, Kubeflow, Seldon, AWS, Container Registry, Secret Manager*

We have already built some really impressive ML pipelines in the previous lessons. In this lesson, we will switch our MLOps stack to run all of our pipelines on a dynamically-scalable [Kubernetes](https://kubernetes.io/) cluster in the cloud.

To do so, we will use [Kubeflow](https://www.kubeflow.org/) to orchestrate our pipelines on the Kubernetes cluster, deploy our models using [Seldon](https://www.seldon.io/), and run and store everything in the [Amazon AWS](https://aws.amazon.com/) cloud.

![Integrations](_assets/4-1/integrations.png)

You will need several new integrations for this notebook. Install them by running the following cell if you have not done so, which will also restart the kernel of your notebook.

In [ ]:
!zenml integration install sklearn mlflow evidently kubeflow seldon s3 aws -y

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

Let's also ignore warning messages and turn on autoreload for our code.

In [ ]:
from absl import logging as absl_logging
import warnings

warnings.filterwarnings("ignore")
absl_logging.set_verbosity(-10000)

%load_ext autoreload
%autoreload 2

## Production-Grade MLOps Stacks

Let's get started by defining our new MLOps stack. Here is what we want to do and the tools we need to achieve that:
- Run our entire pipeline as a Kubernetes cluster in the cloud, using [Kubeflow](https://www.kubeflow.org/) on [Amazon Elastic Kubernetes Service (EKS)](https://aws.amazon.com/eks/) with [Amazon Elastic Container Registry (ECR)](https://aws.amazon.com/ecr/) for pipeline orchestration,
- Store our artifacts and metadata in the cloud, using [Amazon S3](https://aws.amazon.com/s3/) buckets and [Kubeflow ML Metadata](https://www.kubeflow.org/docs/components/pipelines/concepts/metadata/),
- Deploy our models at scale with a [Seldon Core](https://www.seldon.io/solutions/open-source-projects/core) model deployer.

An overview of our desired pipeline and stack is shown below:

![AWS Kubeflow Stack Overview](_assets/4-1/aws_kubeflow_stack.png)

Since ZenML is tool and infrastructure agnostic, we will be able to reuse all of the ML pipeline code we have built throughout the previous chapters.

To run this code on the new production stack, we still need to do two things:

1. Set up all the necessary cloud resources,
2. Register a new ZenML stack with those cloud resources.

## Set Up Cloud Resources

Before we continue, you will need to set up EKS, ECR, and S3. Please refer to the [ZenML Cloud Guide](https://docs.zenml.io/stack-deployment-guide/overview) for instructions on how to do this.

After provisioning the hardware, you also need to:
- [Install Kubeflow](https://www.kubeflow.org/docs/components/pipelines/installation/standalone-deployment/#deploying-kubeflow-pipelines) on your cluster, as shown in the [Kubeflow Orchestrator page](https://docs.zenml.io/mlops-stacks/orchestrators/kubeflow#how-to-deploy-it) in our docs.
- [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html) on your cluster. We have prepared detailed instructions on how to install Seldon on an EKS cluster [here](https://github.com/zenml-io/zenml/tree/main/examples/seldon_deployment#installing-seldon-core-eg-in-an-eks-cluster).

Additionally, you will need [Docker](https://docs.docker.com/get-docker/), [Kubernetes](https://kubernetes.io/docs/tasks/tools/), and [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html) installed on your local machine.

## Register your AWS Kubeflow Stack

To register your stack with ZenML, define the following variables according to your cloud setup:

- **AWS_REGION:** The region where you have provisioned your cloud resources
- **AWS_EKS_CLUSTER:** The name of your EKS cluster
- **ECR_REGISTRY_URL:** The URL to your ECR registry
- **S3_BUCKET_URL:** The URL to your S3 bucket
- **KUBEFLOW_NAMESPACE:** Namespace of your Kubeflow cluster
- **KUBE_CONTEXT:** Name of your Kubernetes context

In [ ]:
# TODO: Replace the following with your own configuration.

AWS_REGION = "us-east-1"
AWS_EKS_CLUSTER = "zenhacks-cluster"
ECR_REGISTRY_NAME = "715803424590.dkr.ecr.us-east-1.amazonaws.com"
S3_BUCKET_NAME = "s3://zenbytes-bucket"
KUBEFLOW_NAMESPACE = "kubeflow"
KUBE_CONTEXT = "zenml-eks"

Next, set up local access to the AWS EKS cluster and the AWS ECR registry by executing the following commands:

In [ ]:
# Point Docker to the ECR registry
!aws ecr get-login-password --region {AWS_REGION} | docker login --username AWS --password-stdin {ECR_REGISTRY_NAME}

# Create a Kubernetes configuration context that points to the EKS cluster
!aws eks --region {AWS_REGION} update-kubeconfig --name {AWS_EKS_CLUSTER} --alias {KUBE_CONTEXT}

Then, run the following cell to extract the base URL that Seldon Core will use to expose all model servers:

In [ ]:
INGRESS_HOST = ! echo $(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].hostname}')

Finally, register the ZenML Stack with the following command:

In [ ]:
# Register container registry
!zenml container-registry register ecr_registry --flavor=default --uri={ECR_REGISTRY_NAME}

# Register orchestrator (Kubeflow on AWS)
!zenml orchestrator register eks_orchestrator --flavor=kubeflow --kubernetes_context={KUBE_CONTEXT} --synchronous=True

# Register metadata store and artifact store
!zenml metadata-store register kubeflow_metadata_store --flavor=kubeflow
!zenml artifact-store register s3_store --flavor=s3 --path={S3_BUCKET_NAME}

# Register the Seldon Core model deployer (Seldon on AWS)
!zenml model-deployer register eks_seldon --flavor=seldon --kubernetes_context={KUBE_CONTEXT} --kubernetes_namespace={KUBEFLOW_NAMESPACE} --base_url=http://{INGRESS_HOST[0]} --secret=s3_store

# Register a secret manager
!zenml secrets-manager register aws_secret_manager --flavor=aws --region_name={AWS_REGION}

!zenml data-validator register evidently_validator --flavor=evidently

# Register the aws_kubeflow_stack
!zenml stack register aws_kubeflow_stack -m kubeflow_metadata_store -a s3_store -o eks_orchestrator -c ecr_registry -d eks_seldon -x aws_secret_manager -dv evidently_validator

In [ ]:
!zenml stack set aws_kubeflow_stack
!zenml stack describe

## Register Seldon and AWS Secrets in ZenML

Lastly, we still need to set up ZenML Secrets to give Seldon Core access to the AWS S3 artifact store in the configured namespace. We can do this with ZenML using the `zenml secret register` command.

**Note:** this is based on the assumption that Seldon Core is running in an EKS cluster that already has IAM access enabled and doesn't need any explicit AWS credentials. For more information on setting up ZenML secrets for Seldon Core, please see the [Managing Seldon Core Credentials](https://github.com/zenml-io/zenml/blob/main/examples/seldon_deployment/README.md#managing-seldon-core-credentials) section in our [Seldon Core Continuous Deployment Example](https://github.com/zenml-io/zenml/blob/main/examples/seldon_deployment/README.md).

For the IAM access case, you can run this command to create the secret:

`zenml secret register -s seldon_s3 s3_store`
, and only set the `rclone_config_s3_env_auth` key to `True`. However, we cannot do this in a Jupyter Notebook because interactive CLI commands are not supported, so we'll do it programmatically:

In [ ]:
SECRET_NAME = "zenbytes-s3-store"

from zenml.repository import Repository
from zenml.integrations.seldon.secret_schemas import SeldonS3SecretSchema
from botocore.exceptions import ClientError

secrets_manager = Repository().active_stack.secrets_manager
try:
    secrets_manager.get_secret(SECRET_NAME)
except ClientError as e:
    secret = SeldonS3SecretSchema(
        name=SECRET_NAME, rclone_config_s3_env_auth=True
    )
    secrets_manager.register_secret(secret)

!zenml secret get {SECRET_NAME}

In [ ]:
!zenml stack up

The output will indicate the URL that we can access to view Kubeflow pipelines locally (e.g., [http://localhost:8080/](http://localhost:8080/)).

## Running on the AWS Kubeflow Stack

We can run the code we wrote with ZenML throughout the previous lessons with any infrastructure and tooling, so once we have our stack set up and configured, we can directly execute our code on it without further changes.

In the background, ZenML automatically detects that the stack has changed. Instead of running your pipeline locally, it will build a Docker image, push it to the container registry with your requirements, and deploy the pipeline with that image on Kubeflow Pipelines. This whole deployment process is usually very painful, but ZenML abstracts all the complexity away.

Let's try it out now! Execute the following cell to run and deploy our digits training pipeline on the new stack. This might take a few minutes to build and push the Docker image. After that, you will see your pipeline run in the cloud!

<div class="alert alert-block alert-info">
    <b>Note:</b> Currently, we cannot run pipelines defined within a Jupyter notebook cell on the cloud, so we need to define and run our pipeline within a Python script instead. We have already defined the corresponding code in `run.py`. Execute the following cell to run our pipeline in the cloud:
</div>

In [ ]:
!python run.py --deploy # --interval-second=300

After the previous command has finished, you should now be able to see the AWS Kubeflow pipeline among ZenMLs list of served models:

In [ ]:
!zenml served-models list

We can now interact with this model as usual, e.g., within another inference pipeline.

Finally, to switch back to our previous local setup, all we have to do is run `zenml stack set default`, then all of our pipelines will be run locally again. Similarly, running `zenml stack set aws_kubeflow_stack` afterward will switch back to the AWS Kubeflow cloud stack. Transitioning from one MLOps stack to another has never been that easy!

Throughout this lesson, you have successfully transitioned to a highly-scalable cloud infrastructure! This really is no small feat, great job!!!

## Cleanup

Once you are done running pipelines with the AWS stack, you can run the following command to stop the Seldon Core model server and the local daemons:

In [ ]:
!zenml served-models delete <UUID-of-model-server>  # see output of `zenml served-models list` above
!zenml stack down -y